<a href="https://colab.research.google.com/github/PedroTonus/praticasGSI073/blob/main/GSI073_aula0_luong_attention.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Preparação dos dados

Esta tarefa é inverter sequências de caracteres. Exemplo: **aabcd** em **dcbaa**.


In [ ]:
import torch
import torch.nn as nn
import random
import torch.nn.functional as F

chars = list("abcd ")
vocab = {ch: i for i, ch in enumerate(chars)} # Cada letra, ganha um número
inv_vocab = {i: ch for ch, i in vocab.items()}# Tabela de decodificação
vocab_size = len(vocab)

def encode(s): # Codifica letras em números
    return torch.tensor([vocab[c] for c in s], dtype=torch.long)

def decode(t): # Decodifica números em letras
    return ''.join(inv_vocab[int(x)] for x in t)

def random_seq(n=5): # Cria novas sequências
    return ''.join(random.choice(chars[:-1]) for _ in range(n))

# Gerar dados
pairs = [(encode(s), encode(s[::-1])) for s in [random_seq() for _ in range(50000)]]

max_len = max(len(x) for x, _ in pairs) # pega maior sequência

def pad(x):  # Preenche conjunto de dados em pad no último índice
    return torch.cat([x, torch.tensor([vocab[' ']] * (max_len - len(x)))], dim=0)

inputs = torch.stack([pad(x) for x, _ in pairs])
targets = torch.stack([pad(y) for _, y in pairs])

train_ds = torch.utils.data.TensorDataset(inputs, targets)
train_dl = torch.utils.data.DataLoader(train_ds, batch_size=128, shuffle=True)

device = 'cuda' if torch.cuda.is_available() else 'cpu'

## Veja um par

In [ ]:
print(pairs[1])

# Definição do modelo Seq2Seq com GRU

In [ ]:
class Encoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True)

    def forward(self, x):
        x = self.embed(x)
        outputs, h = self.gru(x)
        return outputs, h   # <--- ESSENCIAL

In [ ]:
class LuongAttention(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, decoder_hidden, encoder_outputs):
        """
        decoder_hidden: (B, 1, H)
        encoder_outputs: (B, S, H)

        Retorna:
          context: (B, 1, H)
          attn_weights: (B, 1, S)
        """

        # score = h_t · h_s^T
        # (B, 1, H) x (B, H, S) -> (B, 1, S)
        attn_scores = torch.bmm(decoder_hidden, encoder_outputs.transpose(1, 2))

        attn_weights = F.softmax(attn_scores, dim=-1)  # normaliza nos steps da source

        # context = soma ponderada
        # (B, 1, S) x (B, S, H) -> (B, 1, H)
        context = torch.bmm(attn_weights, encoder_outputs)

        return context, attn_weights

In [ ]:
class Decoder(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True)
        self.attn = LuongAttention()

        # Luong concat: concatena hidden + context
        self.fc = nn.Linear(hidden_size * 2, vocab_size)

    def forward(self, x, h, encoder_outputs):
        """
        x: tokens anteriores corretos  (B, T)
        h: estado inicial do decoder   (1, B, H)
        encoder_outputs: todos os h_s  (B, S, H)
        """
        x = self.embed(x)  # (B, T, E)

        outputs = []
        seq_len = x.size(1)
        hidden = h

        for t in range(seq_len):
            inp = x[:, t:t+1]  # (B, 1, E)

            out_t, hidden = self.gru(inp, hidden)   # out_t: (B,1,H)

            # Atenção
            context, attn_w = self.attn(out_t, encoder_outputs)

            # concatenação [out_t ; context]
            combined = torch.cat([out_t, context], dim=-1)

            logits = self.fc(combined)  # (B,1,V)
            outputs.append(logits)

        outputs = torch.cat(outputs, dim=1)  # (B, T, V)
        return outputs, hidden


In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, tgt):
        encoder_outputs, h = self.encoder(src)
        logits, _ = self.decoder(tgt[:, :-1], h, encoder_outputs)
        return logits

# Código para usar o modelo treinado: inferência

In [ ]:
def decode_step(decoder, token, h, encoder_outputs):
    """
    Executa um passo de decodificação:
    - token: tensor (B,1)
    - h: estado oculto do decoder (1,B,H)
    - encoder_outputs: (B,S,H)
    """
    logits, h = decoder(token, h, encoder_outputs)  # (B,1,V)
    next_token = logits[:, -1, :].argmax(-1, keepdim=True)  # (B,1)
    return next_token, h


def predict(model, seq, max_len=10):
    model.eval()
    with torch.no_grad():
        # codifica entrada
        src = pad(encode(seq)).unsqueeze(0).to(device, dtype=torch.long)

        # encoder agora retorna (encoder_outputs, h)
        encoder_outputs, h = model.encoder(src)

        # token inicial (ex: espaço ou <sos>)
        token = torch.tensor([[vocab[' ']]], dtype=torch.long, device=device)

        seq_invertida = []
        for _ in range(max_len):
            token, h = decode_step(model.decoder, token, h, encoder_outputs)
            seq_invertida.append(token.item())

        return decode(seq_invertida)


# Preparação para treino

In [ ]:
emb_size = 32
hidden_size = 64
encoder = Encoder(vocab_size, emb_size, hidden_size)
decoder = Decoder(vocab_size, emb_size, hidden_size)
model = Seq2Seq(encoder, decoder).to(device)

loss_fn = nn.CrossEntropyLoss(ignore_index=vocab[' ']) # ignora o pad: " "
opt = torch.optim.Adam(model.parameters(), lr=1e-3)

# Execução do treino

In [ ]:
for epoch in range(10):
    model.train()
    total_loss = 0
    for xb, yb in train_dl:
        xb, yb = xb.to(device, dtype=torch.long), yb.to(device, dtype=torch.long)
        opt.zero_grad()
        logits = model(xb, yb)
        loss = loss_fn(logits.reshape(-1, vocab_size), yb[:, 1:].reshape(-1))
        loss.backward()
        opt.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}: loss={total_loss/len(train_dl):.4f}")

# Vamos testar

In [ ]:
for _ in range(10):
    s = random_seq()
    pred = predict(model, s, max_len=len(s))
    print(f"{s} -> {pred}")


# Exercício
Compare o resultado do uso do encoder-decoder com atenção com o encoder-decoder sem atenção.

In [ ]:
class DecoderWithoutAttention(nn.Module):
    def __init__(self, vocab_size, emb_size, hidden_size):
        super().__init__()
        self.embed = nn.Embedding(vocab_size, emb_size)
        self.gru = nn.GRU(emb_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, vocab_size) # No concatenation with context

    def forward(self, x, h, encoder_outputs): # encoder_outputs is not used here
        x = self.embed(x)  # (B, T, E)

        outputs = []
        seq_len = x.size(1)
        hidden = h

        for t in range(seq_len):
            inp = x[:, t:t+1]  # (B, 1, E)
            out_t, hidden = self.gru(inp, hidden)   # out_t: (B,1,H)

            # No attention mechanism, just feed GRU output to FC layer
            logits = self.fc(out_t)  # (B,1,V)
            outputs.append(logits)

        outputs = torch.cat(outputs, dim=1)  # (B, T, V)
        return outputs, hidden


In [ ]:
class Seq2SeqWithoutAttention(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, src, tgt):
        # For a model without attention, encoder_outputs might not be explicitly passed to the decoder
        # But the DecoderWithoutAttention is still designed to accept it (though it ignores it).
        encoder_outputs, h = self.encoder(src)
        logits, _ = self.decoder(tgt[:, :-1], h, encoder_outputs) # Pass encoder_outputs, but it's ignored by DecoderWithoutAttention
        return logits


In [ ]:
print("\n--- Treinando o modelo SEM atenção ---")
encoder_no_attn = Encoder(vocab_size, emb_size, hidden_size)
decoder_no_attn = DecoderWithoutAttention(vocab_size, emb_size, hidden_size)
model_no_attn = Seq2SeqWithoutAttention(encoder_no_attn, decoder_no_attn).to(device)

opt_no_attn = torch.optim.Adam(model_no_attn.parameters(), lr=1e-3)

for epoch in range(10):
    model_no_attn.train()
    total_loss_no_attn = 0
    for xb, yb in train_dl:
        xb, yb = xb.to(device, dtype=torch.long), yb.to(device, dtype=torch.long)
        opt_no_attn.zero_grad()
        logits = model_no_attn(xb, yb)
        loss = loss_fn(logits.reshape(-1, vocab_size), yb[:, 1:].reshape(-1))
        loss.backward()
        opt_no_attn.step()
        total_loss_no_attn += loss.item()
    print(f"Epoch {epoch+1} (No Attention): loss={total_loss_no_attn/len(train_dl):.4f}")


In [ ]:
def predict_no_attention(model, seq, max_len=10):
    model.eval()
    with torch.no_grad():
        src = pad(encode(seq)).unsqueeze(0).to(device, dtype=torch.long)
        encoder_outputs, h = model.encoder(src)
        token = torch.tensor([[vocab[' ']]], dtype=torch.long, device=device)
        seq_invertida = []
        for _ in range(max_len):
            # DecoderWithoutAttention ignores encoder_outputs
            logits, h = model.decoder(token, h, encoder_outputs) # Pass encoder_outputs, but it's ignored by DecoderWithoutAttention
            next_token = logits[:, -1, :].argmax(-1, keepdim=True)  # (B,1)
            seq_invertida.append(next_token.item())
        return decode(seq_invertida)


In [ ]:
print("\n--- Comparação dos resultados ---")
for _ in range(10):
    s = random_seq()
    pred_attn = predict(model, s, max_len=len(s)) # Original model with attention
    pred_no_attn = predict_no_attention(model_no_attn, s, max_len=len(s)) # New model without attention
    print(f"Original: {s} -> Com Atenção: {pred_attn} / Sem Atenção: {pred_no_attn}")


### Comparação dos Modelos com e Sem Atenção

Após o treinamento e a comparação dos dois modelos, podemos observar as seguintes diferenças:

*   **Modelo com Atenção:** Este modelo (com a classe `Decoder` original) foi capaz de inverter as sequências de caracteres de forma altamente precisa, muitas vezes produzindo a sequência invertida exata. Isso ocorre porque o mecanismo de atenção permite que o decodificador 'olhe' para as partes mais relevantes da sequência de entrada a cada passo de decodificação, ajudando-o a construir a sequência de saída correta.

*   **Modelo Sem Atenção:** O modelo sem atenção (utilizando a classe `DecoderWithoutAttention`) demonstrou uma performance significativamente inferior. As previsões deste modelo frequentemente resultaram em sequências incorretas, muitas vezes com a repetição de caracteres ou a geração de padrões que não correspondiam à inversão esperada. A ausência do mecanismo de atenção impede que o decodificador foque em partes específicas da entrada, levando a uma perda de informação crucial e dificuldade em mapear corretamente a entrada para a saída.

**Conclusão:** A comparação reforça a importância crítica do mecanismo de atenção em arquiteturas Seq2Seq para tarefas como a inversão de sequências. A atenção melhora drasticamente a capacidade do modelo de reter e utilizar informações contextuais da sequência de entrada durante o processo de decodificação, resultando em previsões muito mais precisas e robustas.